### Задание 1. Очень усердные ученики

#### Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц **правильно** решил 20 горошин.

#### Задача

Дана таблица **default.peas**:

| Название атрибута | Тип атрибута | Смысловое значение                       |
| :---------------- | :----------- | :--------------------------------------- |
| st_id             | int          | ID ученика                               |
| timest            | timestamp    | Время решения карточки                   | 
| correct           | bool         | Правильно ли решена горошина?            |      
| subject           | text         | Дисциплина, в которой находится горошина |  

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов. *NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.*

#### Решение

In [1]:
# импортируем библиотеки

import pandahouse as ph
import pandas as pd

In [2]:
# объявляем параметры подключения

connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                           'database':'default',
                           'user':'student', 
                           'password':'dpo_python_2020'
                          }

In [13]:
# пишем запрос, и получаем данные из clickhouse в pandas dataframe

query = """
SELECT 
   count(distinct st_id) as hardworking_students
FROM
   (SELECT 
      st_id, 
      sum(correct) as correct
    FROM default.peas
    GROUP BY st_id
    HAVING correct >= 20)
"""

hard_students = ph.read_clickhouse(query, connection=connection_default)

#### Ответ

In [21]:
print('Количество очень усердных студентов:', hard_students['hardworking_students'][0])

Количество очень усердных студентов: 136


### Задание 2. Оптимизация воронки

#### Условие 

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

#### Задача

Даны таблицы: **default.peas** (см. выше), **default.studs**:

| Название атрибута | Тип атрибута | Смысловое значение                       |
| :---------------- | :----------- | :--------------------------------------- |
| st_id             | int          | ID ученика                               |
| test_grp          | text         | Метка ученика в данном эксперименте      | 

и **default.final_project_check**:

| Название атрибута | Тип атрибута | Смысловое значение                       |
| :---------------- | :----------- | :--------------------------------------- |
| st_id             | int          | ID ученика                               |
| sale_time         | timestamp    | Время покупки                            | 
| money             | int          | Цена, по которой приобрели данный курс   |      
| subject           | text         |                                          | 

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

* ARPU 
* ARPAU 
* CR в покупку 
* СR активного пользователя в покупку 
* CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.

**Активным** считается пользователь, за все время решивший **больше** 10 задач правильно в любых дисциплинах.

**Активным** по математике считается пользователь, за все время решивший **2 или больше** задач правильно по математике.

#### Решение

In [3]:
query = """
WITH unique_users as ( 
    SELECT  
       st_id,
       test_grp  
    FROM default.studs
    GROUP BY 
       st_id,
       test_grp),
    total_data as (
    SELECT  
       u.st_id as st_id,
       test_grp,
       revenue,
       revenue_math,
       correct_answers,
       correct_math_answers
    FROM unique_users as u
    LEFT JOIN 
            (
            SELECT
               st_id,
               sum(money) as revenue,
               sumIf(money, subject='Math') as revenue_math
            FROM default.final_project_check
            GROUP BY st_id
            ) as r
    ON u.st_id = r.st_id
    LEFT JOIN 
            (
            SELECT  
               st_id,
               sum(correct) as correct_answers,
               sumIf(correct, subject='Math') as correct_math_answers
            FROM default.peas
            GROUP BY st_id
            ) as c
    ON u.st_id = c.st_id) 
SELECT 
   test_grp,
   round(sum(revenue) / count(st_id), 2) as ARPU,
   round(sumIf(revenue, correct_answers > 10) / countIf(correct_answers > 10), 2) as ARPAU,
   round(countIf(revenue > 0) / count(st_id) * 100, 2) as CR,
   round(countIf(revenue > 0 and correct_answers > 10) / countIf(correct_answers > 10) * 100, 2) as CR_active_users,
   round(countIf(revenue_math > 0 and correct_math_answers > 2) / countIf(correct_math_answers > 2) * 100, 2) as CR_active_math
FROM total_data   
GROUP BY test_grp
"""

metrics = ph.read_clickhouse(query, connection=connection_default)

#### Ответ

In [4]:
metrics

,test_grp,ARPU,ARPAU,CR,CR_active_users,CR_active_math
0,control,4540.98,10393.70,4.92,11.02,6.12
1,pilot,11508.47,29739.58,10.85,26.04,9.52
